# Processing new data sets

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense

## Prepping data

In [25]:
# gather data set and split in x and y
def read_dataset(filename):
    df = pd.read_csv(filename)
    df = df.loc[:, ~df.columns.str.match('Unnamed')]
    df = df.loc[:, ~df.columns.str.match('operator')]
    df = df.loc[:, ~df.columns.str.match('campaign')]
    df = df.loc[:, ~df.columns.str.match('Timestamp')]
    df = df.replace('---', 3)
    df = df.replace('Yes', 1)
    df = df.replace('5G EN-DC', 1)
    df = df.replace('LTE', 0)

    X, y = df.drop('Mode', axis=1), df['Mode']
    return X, y

# splitting the data into time series that overlap
def split_time_series(X, y):
    tss = TimeSeriesSplit(n_splits=2)

    for train_index, test_index in tss.split(X):
        X_train, X_test = np.array(X.iloc[train_index, :]), np.array(X.iloc[test_index,:])
        y_train, y_test = np.array(y.iloc[train_index]), np.array(y.iloc[test_index])

    # reshaping y to match dimentions of X
    y_train = y_train.reshape(len(y_train), 1)
    y_test = y_test.reshape(len(y_test), 1)
    # horizontally stack columns
    trainingset = np.hstack((X_train, y_train))
    testset = np.hstack((X_test, y_test))

    return trainingset, testset

def series_split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check to see if we are bwyond the data set
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def parallel_split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check to see if we are beyond the data set
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def vertical_handover(y):
    new_y = []
    for seq in y:
        if 0 in seq and 1 in seq:
            new_y.append(1)
        else:
            new_y.append(0)
    return np.array(new_y)

def accuracy(y_pred, y_true):
    return np.mean(y_pred == y_true)

def precision_recall(y_pred, y_true):
    # Initialize true positives (TP), false positives (FP), and false negatives (FN)
    tp, fp, fn =  0, 0, 0

    # Loop through true and predicted labels to count TP, FP, and FN
    for yt, yp in zip(y_true, y_pred):
        if yt == 1 and yp == 1:
            tp += 1  # True Positive
        elif yt == 0 and yp == 1:
            fp += 1  # False Positive
        elif yt == 1 and yp == 0:
            fn += 1  # False Negative

    print(tp, fp, fn)

    # Precision calculation
    if tp + fp == 0:
        precision = 0
    else:
        precision = tp / (tp + fp)

    # Recall calculation
    if tp + fn == 0:
        recall = 0
    else:
        recall = tp / (tp + fn)

    return precision, recall

In [26]:
X, y = read_dataset('datasets/Active_Ihne.csv')
train, test = split_time_series(X, y)

n_steps_in, n_steps_out = 50, 30

X_train, y_train = series_split_sequences(train, n_steps_in, n_steps_out)
X_test, y_test = series_split_sequences(test, n_steps_in, n_steps_out)
y_test_changed = vertical_handover(y_test)

n_features = X_train.shape[2]

losses, accuracies, recalls, precisions = [], [], [], []

for _ in range(3):
    model = Sequential()
    model.add(LSTM(units=500, activation='relu',return_sequences=True, input_shape=(n_steps_in, n_features)))
    model.add(LSTM(units=500, activation='relu'))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')

    model.fit(X_train, y_train, epochs=300, verbose=0)

    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)
    y_pred = vertical_handover(y_pred)
    loss, acc = model.evaluate(X_test, y_test), accuracy(y_pred, y_test_changed)
    prec, rec = precision_recall(y_pred, y_test_changed)
    losses.append(loss)
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)

print(f'Loss Mean: {np.mean(losses):.3f}')
print(f'Loss Standard Deviation: {np.std(losses):.3f}')
print(f'Accuracy Mean: {np.mean(accuracies):.3f}')
print(f'Accuracy Standard Deviation: {np.std(accuracies):.3f}')
print(f'Precision Mean: {np.mean(precisions):.3f}')
print(f'Precision Standard Deviation: {np.std(precisions):.3f}')
print(f'Recall Mean: {np.mean(recalls):.3f}')
print(f'Recall Standard Deviation: {np.std(recalls):.3f}')

* Se på tidsgapet og sammenligne kurve i tidslinjen (mer og mindre pings)
* Se på PCI primary og secondary
* Skal ha en base line